AB测试项目：确定产品售价

项目介绍
公司XYZ以39美元的价格出售软件由于一段时间以来收入一直保持稳定，因此产品总监决定进提高价格测试，希望能够将收入增加。在实验中，有66％的用户看到的是旧价格（$39），而随机抽样的33％的用户则看到了较高的价格（$59）。

该测试已经运行了一段时间，产品经理想要了解测试的进行方式，以及为所有用户提高价格是否可行。

项目要解决的问题
1. 公司应该把软件售价定为39美元还是59美元?


2. 产品总监有兴趣全面了解用户行为，尤其关注可提高转化率的可行方法。通过这些数据，你的主要发现是什么?


3. 产品总监认为测试已经运行了太长时间，认为他们应该能够在更短的时间内得到统计上显著的结果。你同意吗?如果是你，过了多少天你会停止测试?请解释为什么。

3. 项目用到的数据集:test_results 和user_table
test_results:测试数据集
* user_id:用户ID
* timestamp：用户第一次点击公司XYZ网页的日期和时间。以用户本地时间为准
* source:引导用户访问网站的营销渠道。
* device:用户设备，mobile/web
* operative_system:用户操作系用，windows,linux,mac->web,android/ios->mobile, other->none 
* test:用户是在实验组(即1 ->更高的价格)还是在控制组(0 ->旧的/更低的价格)
* price：用户看到的价格。和test相匹配
* coverted:用户是否转换(1 ->购买软件)或没有(0 ->离开网站没有购买)。

user_table:用户信息数据集
* user_id:用户ID
* city:用户所在的城市。来自用户ip地址
* country: 用户所在国家
* lat:城市纬度，与用户城市匹配
* long:城市经度，与与用户城市匹配

4.1检查两个数据集

In [55]:
import pandas as pd 
import numpy as np

test=pd.read_csv('test_results.csv')
test.head()

,user_id,timestamp,source,device,operative_system,test,price,converted
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0


In [56]:
#检查test数据集是否有重复
print(len(test)==test['user_id'].nunique())

True


In [57]:
user=pd.read_csv('user_table.csv')
user.head()

,user_id,city,country,lat,long
0,510335,Peabody,USA,42.53,-70.97
1,89568,Reno,USA,39.54,-119.82
2,434134,Rialto,USA,34.11,-117.39
3,289769,Carson City,USA,39.15,-119.74
4,939586,Chicago,USA,41.84,-87.68


In [58]:
#检查user数据集是否有重复
print(len(user)==user['user_id'].nunique())

True


In [59]:
#再检查两个数据集用户数是否一样
print(len(user['user_id']) - len(test['user_id']))

-41184


In [60]:
#User数据集中有一些用户id丢失了，数量非常的少（对比总数据），所以我们可以用内部连接方法去掉那些值。
data=test.merge(user, on=['user_id'])
data.head()

,user_id,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0,Parma,USA,41.38,-81.73
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0,Fayetteville,USA,35.07,-78.90
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0,Fishers,USA,39.95,-86.02


4.2 检查实验组和控制组里的price是否一致

In [61]:
data.groupby(['price']).test.value_counts()

price  test
39     0       176241
       1          135
59     1        99053
       0          187
Name: test, dtype: int64

从这里可以看出价格有小部分划分错误，有39的被划分到了实验组，59的被划分到了控制组

In [62]:
#把这一小部分错误划分的数据集删除

#把实验组中价值为39的行删除
data.drop(index=(data.loc[(data['test']==1)&(data['price']==39)].index),inplace=True)

#把控制组中价值为59的行删除
data.drop(index=(data.loc[(data['test']==0)&(data['price']==59)].index),inplace=True)

In [63]:
data.head()

,user_id,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0,Parma,USA,41.38,-81.73
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0,Fayetteville,USA,35.07,-78.90
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0,Fishers,USA,39.95,-86.02


4.3 检查实验组和控制组的组成成分是否相似，也就是检查ABtest是否有效随机化

方法一: 卡方检验
主要是用在某个变量（特征）值是不是和应变量有显著关系，也就是看某个变量是否独立。实际观测值与理论统计值之间的偏离程度越大，卡方值越大。

原假设和备选假设：

H0:原假设是独立变量，对应变量没有影响，实际观测频率和理论频率一致

H1:原假设并非独立变量，对应变量有影响，实际观测频率和理论频率不同。

In [64]:
#选取变量创建数据集用于检验实验组和控制组的组成成分
Fdata=data[['user_id','source','device','operative_system','test','city']]

Fdata.set_index(['user_id'],inplace=True)
Fdata.head()

,source,device,operative_system,test,city
user_id,,,,,
604839,ads_facebook,mobile,iOS,0,Buffalo
624057,seo-google,mobile,android,0,Lakeville
317970,ads-bing,mobile,android,0,Parma
685636,direct_traffic,mobile,iOS,1,Fayetteville
820854,ads_facebook,web,mac,0,Fishers


In [65]:
#将分类型变量转换成数值label
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
source_label_encoder = LabelEncoder()
device_label_encoder = LabelEncoder()
os_label_encoder = LabelEncoder()
city_label_encoder = LabelEncoder()

Fdata["source"] = source_label_encoder.fit_transform(Fdata.source)
Fdata["device"] = device_label_encoder.fit_transform(Fdata.device)
Fdata["operative_system"] = os_label_encoder.fit_transform(Fdata.operative_system)
Fdata["city"] = city_label_encoder.fit_transform(Fdata.city)
Fdata.head()

,source,device,operative_system,test,city
user_id,,,,,
604839,3,0,1,0,109
624057,8,0,0,0,433
317970,0,0,0,0,612
685636,5,0,1,1,278
820854,3,1,3,0,281


In [66]:
#方法一：使用sklearn中特征选择里的chi2
from sklearn.feature_selection import chi2,f_classif

colnames = ["source","device","operative_system","city"]
ch2values,pvalues = chi2(Fdata.loc[:,colnames],Fdata["test"])
pd.DataFrame({'chi2_value':ch2values,'pvalue':pvalues},index = colnames).sort_values(by='pvalue')

,chi2_value,pvalue
city,189.704637,3.688100e-43
operative_system,69.103650,9.342229e-17
device,38.695245,4.954146e-10
source,4.432809,3.525445e-02


假设alpa=0.05,以上所有变量的p值都小于alpha,可以拒绝原假设，都不是独立变量，都对实验组和控制组的划分有影响，由此可知ABtest是无效随机化，有选择性偏差

In [67]:
#方法二：使用scipy.stats 中chi2_contingency
from scipy.stats import chi2_contingency
from scipy.stats import chi2

for i in colnames:
    x=pd.crosstab(Fdata.loc[:,i],Fdata.loc[:,'test'])
    
    # stat卡方统计值，p：P_value，dof 自由度，expected理论频率分布
    stat,p,dof,expected=chi2_contingency(x)
    
    #选取95%置信度
    prob=0.95 
    alpha = 1-prob
    print('p=%.3f'%p)
    
    if p<alpha:
        print('拒绝 H0:认为%s变量有显著性影响,则拒绝原假设,变量不独立'%i)
    else:
        print('不拒绝H0:认为%s变量没有显著性影响,则接受原假设,变量独立'%i )
    print()

p=0.001
拒绝 H0:认为source变量有显著性影响,则拒绝原假设,变量不独立

p=0.000
拒绝 H0:认为device变量有显著性影响,则拒绝原假设,变量不独立

p=0.000
拒绝 H0:认为operative_system变量有显著性影响,则拒绝原假设,变量不独立

p=0.000
拒绝 H0:认为city变量有显著性影响,则拒绝原假设,变量不独立



In [68]:
#查看source变量在实验组和控制组的分布
data.groupby('test')['source'].apply(lambda s: s.value_counts(normalize=True)).unstack()

,ads-bing,ads-google,ads-yahoo,ads_facebook,ads_other,direct_traffic,friend_referral,seo-bing,seo-google,seo-other,seo-yahoo,seo_facebook
test,,,,,,,,,,,,
0,0.071850,0.186495,0.023604,0.167725,0.095460,0.191528,0.064769,0.006548,0.074534,0.029068,0.021703,0.066715
1,0.072557,0.188556,0.024290,0.169283,0.093051,0.189444,0.066429,0.007178,0.070629,0.029469,0.021504,0.067610


In [69]:
#查看device变量在实验组和控制组的分布
data.groupby('test')['device'].apply(lambda s: s.value_counts(normalize=True)).unstack()

,mobile,web
test,,
0,0.594669,0.405331
1,0.578832,0.421168


In [70]:
#查看os变量在实验组和控制组的分布
data.groupby('test')['operative_system'].apply(lambda s: s.value_counts(normalize=True)).unstack()

,windows,iOS,android,mac,other,linux
test,,,,,,
0,0.312998,0.304736,0.238180,0.081008,0.052133,0.010945
1,0.327572,0.294570,0.234905,0.076000,0.049781,0.017173


我们可以看到实验组和对照组的OS分布有很大的不同。

方法二: 决策树

1. 使用test变量作为我们的标签。尝试建立一个模型，将test为0的用户和test为1的用户分开。如果ab测试随机化效果良好，这将是不可能的，因为两组完全相同。如果所有变量相对频率都相同，则没有模型能够将test==1与test==0分开。如果ab测试随机化效果不佳，该模型将设法使用一个给定的变量来分隔两组。


2. 选择一个决策树作为模型。可以清楚地看到哪个变量(如果有的话)被用于分割。这就是ab测试随机化分组失败的地方。

In [71]:
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from graphviz import Source

tree = DecisionTreeClassifier(max_depth=3,class_weight="balanced")
tree.fit(Fdata.drop('test', axis=1), Fdata['test'])

export_graphviz(tree, out_file="tree_test.dot", feature_names=Fdata.drop('test', axis=1).columns, proportion=True, rotate=True)
with open("tree_test.dot") as f:
    dot_graph = f.read()
  
s = Source.from_file("tree_test.dot")
s.view()

'tree_test.dot.pdf'

5.问题一：公司应该把软件售价定为39美元还是59美元?

In [72]:
print(data.groupby('test')['converted'].mean())
#控制组转化率(39)比实验组(59)转换率高

test
0    0.019768
1    0.015446
Name: converted, dtype: float64


In [73]:
#新增一列收入revenue
data['revenue']=data['price']*data['converted']

In [74]:
print(data.groupby('test')['revenue'].mean())
#控制组转化率(39)比实验组(59)的平均收入低

test
0    0.770967
1    0.911330
Name: revenue, dtype: float64


比较两个总体均值：独立样本t检验(大样本，总体方差未知时z~=t)

应用场景：是针对两组不相关样本（各样本量可以相等也可以不相等），检验它们在均值之间的差异。

对于该检验方法而言，我们首先要确定两个总体的方差是否相等，如果不等，先利用levene检验，检验两总体是否具有方差齐性。

原假设和备选假设：

H0:两独立样本的收入均值相等    
H1:两独立样本的收入均值不相等

In [75]:
from scipy import stats
stats.levene(data[data['test'] == 1]['revenue'],data[data['test'] == 0]['revenue'])

LeveneResult(statistic=32.95020185928107, pvalue=9.464878518986399e-09)

p值远小于0.05，两总体不具有方差齐性，需要将equal_val参数设定为“False”。

需注意的情况：如果两总体具有方差齐性，错将equal_var设为False，p值变大; 两总体方差不等时，若没有将equal_var参数设定为False，则函数会默认equal_var为True，这样会低估p值

In [76]:
test = stats.ttest_ind(data[data['test'] == 1]['revenue'],data[data['test'] == 0]['revenue'],equal_var=False)
print(test)

Ttest_indResult(statistic=5.298877264308738, pvalue=1.1666887515733168e-07)


In [77]:
#以上是双侧检验，但我们想要的是单侧检验
test.pvalue/2

5.833443757866584e-08

检验结果显示p远小于0.05，我们拒绝原假设，即认为实验组和控制组收入均值存在显著差异，并且实验组收入的均值显著大于控制组的收入均值。

但是由于实验组和控制组分布不相似，AB测试有选择偏差性，所以价格并不是实验组和控制组之间唯一的差异，所以我们不能将平均收益的增长归因于价格。所以需要重新做一次AB测试，确保实验组和控制组之间的唯一差异是价格不同。

6.问题二：提高转化率的方法?

In [78]:
data.head()

,user_id,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long,revenue
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86,0
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24,0
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0,Parma,USA,41.38,-81.73,0
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0,Fayetteville,USA,35.07,-78.90,0
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0,Fishers,USA,39.95,-86.02,0


In [79]:
data.set_index(['user_id'],inplace=True)

In [80]:
data.head()

,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long,revenue
user_id,,,,,,,,,,,,
604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86,0
624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24,0
317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0,Parma,USA,41.38,-81.73,0
685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0,Fayetteville,USA,35.07,-78.90,0
820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0,Fishers,USA,39.95,-86.02,0


In [81]:
data['timestamp']=pd.to_datetime(data['timestamp'],errors='coerce')